# Import

In [18]:
%cd /home/is/dwipraseetyo-a/NAS_HAI/Project/Qwen2.5-Omni
%pwd

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
print("CUDA_VISIBLE_DEVICES:", os.environ["CUDA_VISIBLE_DEVICES"])

import commons, const_variable
import os, librosa, random, pickle, pydicom, requests, torch, re, json, pydicom
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
from dotenv import load_dotenv

load_dotenv()

def fix_broken_bullet_blocks(text):
    lines = text.splitlines()
    fixed_lines = []
    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Detect pattern:
        # Line i: startswith "*   Something"
        # Line i+1: is "*"
        # Line i+2: startswith "*   SomethingElse"
        if line.startswith("*") and not line == "*" and i + 2 < len(lines):
            next_line = lines[i + 1].strip()
            next_next_line = lines[i + 2].strip()
            if next_line == "*" and next_next_line.startswith("*"):
                # Add current line
                fixed_lines.append(lines[i])
                # Merge next_next_line as continuation bullet
                fixed_lines.append("    " + next_next_line)
                i += 3
                continue

        # If not matching pattern, just add the line
        fixed_lines.append(lines[i])
        i += 1

    return "\n".join(fixed_lines)

/home/ldap-users-2/dwipraseetyo-a/Project/Qwen2.5-Omni
CUDA_VISIBLE_DEVICES: 0,1


# Iterate

In [2]:
model_id = "/home/is/dwipraseetyo-a/NAS_HAI/Project/pretrain/medgemma-4b-it"
auth_key = os.getenv("HG_AUTH_KEY")
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=auth_key,
)
processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
DATA_PATH = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets"
for split in ['train', 'dev', 'test']:
    df = pd.read_csv(f"{DATA_PATH}/metadata_cut_processed.csv.{split}")
    results = []
    paths = []
    images = []
    messages_batch = []

    for now_row in tqdm(df.itertuples(), desc=f"Processing {split},", total=len(df)):
        now_path = now_row.path_file_image
        now_image = commons.crop_and_convert("/home/is/dwipraseetyo-a/NAS_HAI/Datasets/cidrz/" + now_path)
        answer = "This is Tuberculosis" if now_row.ground_truth_tb == 1 else "This is Not Tuberculosis"
        system_prompt = (
            "You are an expert radiologist. Analyze the chest X-ray and return only the following sections:\n\n"
            "**Specific Findings:** Provide a comprehensive and detailed description of all relevant radiographic abnormalities.\n\n"
            "**Differential Diagnosis:** Only if the X-ray not healty and describe the reason.\n\n"
            "**Conclusion:** Summarize your diagnostic impression clearly.\n\n"
            "Do not include any **Disclaimer** or unrelated content."
        ) if now_row.ground_truth_tb == 0 else (
            "You are an expert radiologist. Analyze the chest X-ray and return only the following sections:\n\n"
            "**Key Features:** Highlight radiographic patterns that are characteristic of tuberculosis, explained clearly and in detail.\n\n"
            "**Conclusion:** Summarize your diagnostic impression clearly.\n\n"
            "Do not include any **Disclaimer** or unrelated content."
        )
        message = [
            {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
            {"role": "user", "content": [
                {"type": "text", "text": f"Describe the X-ray." + answer},
                {"type": "image", "image": now_image}
            ]}
        ]
        paths.append(now_path)
        messages_batch.append(message)

    # Process in batches
    batch_size = 24
    for i in tqdm(range(0, len(messages_batch), batch_size), desc="Generating", total=len(range(0, len(messages_batch), batch_size))):
        batch_messages = messages_batch[i:i+batch_size]
        batch_paths = paths[i:i+batch_size]

        inputs = processor.apply_chat_template(batch_messages, add_generation_prompt=True, tokenize=True,
                                               return_dict=True, return_tensors="pt", padding=True).to(model.device, dtype=torch.bfloat16)

        input_lens = [len(ids) for ids in inputs['input_ids']]

        with torch.inference_mode():
            generation = model.generate(**inputs, max_new_tokens=768, do_sample=False)

        # Slice outputs to exclude input prompt
        for j in range(len(batch_messages)):
            output_ids = generation[j][input_lens[j]:]
            decoded = processor.decode(output_ids, skip_special_tokens=True)
            results.append({
                "path_file_image": batch_paths[j],
                "llm_analyze_image": decoded
            })
    pd.DataFrame(results).to_csv(f'datas/reasoning/xray/medgemma_xray.csv.{split}', index=False)

Generating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [03:02<00:00, 15.23s/it]


In [22]:
def crop_and_convert(path_file, max_size=1024):
    array_PIL = Image.open(path_file)
    
    # Resize if needed
    width, height = array_PIL.size
    max_dim = max(width, height)
    
    if max_dim > max_size:
        scale = max_size / max_dim
        new_width = int(width * scale)
        new_height = int(height * scale)
        array_PIL = array_PIL.resize((new_width, new_height), resample=Image.Resampling.LANCZOS)
    
    return array_PIL

In [ ]:
disease_map = {
    "Normal": 0,
    "Tuberculosis": 1,
    "Covid-19": 2,
    "Pneumonia": 3,
    "Other": 3,
}
reverse_map = {v: k for k, v in disease_map.items()}

In [11]:
df['disease'].value_counts()

disease
0    12227
3    10499
1     5386
4     4418
2     3068
Name: count, dtype: int64

In [ ]:
DATA_PATH = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets"
for split in ['train', 'val', 'test']:
    df = pd.read_csv(f"{DATA_PATH}/xray_metadata_llm.csv.{split}")
    results = []
    paths = []
    images = []
    messages_batch = []

    for now_row in tqdm(df.itertuples(), desc=f"Processing {split},", total=len(df)):
        now_path = now_row.path_file
        now_image = crop_and_convert(now_path)
        answer = f"This is {reverse_map[now_row.disease]} Disease"
        system_prompt = (
            "You are an expert radiologist. Analyze the chest X-ray and return only the following sections:\n\n"
            "**Specific Findings:** Provide a comprehensive and detailed description of all relevant radiographic abnormalities.\n\n"
            "**Differential Diagnosis:** Only if the X-ray not healty and describe the reason.\n\n"
            "**Conclusion:** Summarize your diagnostic impression clearly.\n\n"
            "Do not include any **Disclaimer** or unrelated content."
        ) if now_row.disease == 0 else (
            "You are an expert radiologist. Analyze the chest X-ray and return only the following sections:\n\n"
            "**Key Features:** Highlight radiographic patterns that are characteristic of disease, explained clearly and in detail.\n\n"
            "**Conclusion:** Summarize your diagnostic impression clearly.\n\n"
            "Do not include any **Disclaimer** or unrelated content."
        )
        message = [
            {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
            {"role": "user", "content": [
                {"type": "text", "text": f"Describe the X-ray." + answer},
                {"type": "image", "image": now_image}
            ]}
        ]
        paths.append(now_path)
        messages_batch.append(message)

    batch_size = 24
    for i in tqdm(range(0, len(messages_batch), batch_size), desc="Generating", total=len(range(0, len(messages_batch), batch_size))):
        batch_messages = messages_batch[i:i+batch_size]
        batch_paths = paths[i:i+batch_size]

        inputs = processor.apply_chat_template(batch_messages, add_generation_prompt=True, tokenize=True,
                                               return_dict=True, return_tensors="pt", padding=True).to(model.device, dtype=torch.bfloat16)

        input_lens = [len(ids) for ids in inputs['input_ids']]

        with torch.inference_mode():
            generation = model.generate(**inputs, max_new_tokens=768, do_sample=False)

        # Slice outputs to exclude input prompt
        for j in range(len(batch_messages)):
            output_ids = generation[j][input_lens[j]:]
            decoded = processor.decode(output_ids, skip_special_tokens=True)
            results.append({
                "path_file_image": batch_paths[j],
                "llm_analyze_image": decoded
            })

    results_df.to_csv(f"{DATA_PATH}/asdasd.csv.{split}", index=False)
    merged = df.merge(
        results_df.rename(columns={"idx_col": IDX_COL}),
        on=IDX_COL,
        how="left"
    )
    merged.to_csv(f"{DATA_PATH}/metadata_llm.csv.{split}", index=False)
    # pd.DataFrame(results).to_csv(f'datas/reasoning/xray/medgemma_xray.csv.{split}', index=False)

Processing train,:   0%|                                                                                                                                               | 0/35598 [00:00<?, ?it/s]

Processing train,:   0%|                                                                                                                                               | 0/35598 [00:00<?, ?it/s]


In [8]:
df['llm_analyze_image'][0]

'**Key Features:**\n\nThe chest X-ray demonstrates diffuse, bilateral interstitial infiltrates, predominantly in the lower lung zones. These infiltrates appear as hazy opacities, often with a reticular or nodular pattern. There is also evidence of pleural effusions, particularly on the left side, which appear as increased density in the lower lung fields. The heart size appears normal.\n\n**Conclusion:**\n\nThe findings are highly suggestive of COVID-19 pneumonia. The diffuse interstitial infiltrates and pleural effusions are characteristic of this viral infection.\n'

In [ ]:
DATA_PATH = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets"
for split in ['train', 'dev', 'test']:
    df = pd.read_csv(f"{DATA_PATH}/xray_medgemma.csv.{split}")
  
    df_temp = pd.DataFrame(columns=['path_file_image', 'llm_analyze_image'])
    for now_row in tqdm(df.itertuples(), desc=f"Processing {split},", total=len(df)):
        now_path = now_row.path_file_image
        text = now_row.llm_analyze_image

        section_pattern = r'(?:\*\*)?\{?([A-Za-z ]+?)\}?:?(?:\*\*)?\n\n|(?:\.\n\n)?([A-Za-z ]+?):\n\n'
        matches = list(re.finditer(section_pattern, text))
        parsed = []

        start = 0
        for i, match in enumerate(matches):
            title = match.group(1) or match.group(2)
            title = title.strip()

            content_start = match.end()
            content_end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
            content = text[content_start:content_end].strip()

            disclaimer = re.search(r'\n\n\*?\*?Disclaimer:.*', content, re.IGNORECASE | re.DOTALL)
            if disclaimer:
                content = content[:disclaimer.start()].strip()
            parsed.append((title, content))
        
        extracted_data = "\n\n"
        for idx, (title, content) in enumerate(parsed):
            if idx == len(parsed) - 1:
                extracted_data += f"{content.replace('*   ', '')}" + " \n\n"
            else:
                extracted_data += f"* {title}\n{content.replace('*   ', '    *   ')}" + " \n\n"
        
        df_temp.loc[len(df_temp)] = [now_path, fix_broken_bullet_blocks(extracted_data)]
    pd.DataFrame(df_temp).to_csv(f'datas/reasoning/xray/medgemma_xray_formatted.csv.{split}', index=False)

In [ ]:
DATA_PATH = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets"
for split in ['train', 'val', 'test']:
    df = pd.read_csv(f"{DATA_PATH}/xray_medgemma.csv.{split}")
  
    formatted_llm = []
    for now_row in tqdm(df.itertuples(), desc=f"Processing {split},", total=len(df)):
        text = now_row.llm_analyze_image
        text = re.sub(r"\*\*.*?\*\*\n\n", "", str(text), flags=re.DOTALL)
        text = re.sub(r"\n\n.*?\n\n", "", text, flags=re.DOTALL)
        text = text.replace("\n", " ").strip()
        text = re.sub(r"\.\s{2,}([A-Z])", r". \1", text)
        text = re.sub(r"\*\s{2,}", "", text)
        formatted_llm.append(text)
    
    df["formatted_llm_analyze_image"] = formatted_llm
    df.to_csv(f"{DATA_PATH}/xray_medgemma.csv.{split}", index=False)

Processing test,: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5301/5301 [00:00<00:00, 119353.50it/s]


In [28]:
DATA_PATH = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets"
split = "test"
df_reasoning = pd.read_csv(f"{DATA_PATH}/xray_medgemma.csv.{split}")
df_original = pd.read_csv(f"{DATA_PATH}/xray_metadata_llm.csv.{split}")

In [65]:
df_original

,user_id,path_file,disease
0,9MYBHXF7M7CO,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,2
1,ZXP9O6G6H72N,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,2
2,FYYUCI8WSKIA,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,2
3,D9N0U52KL6A5,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,2
4,J0L7VJ00EWJV,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,2
...,...,...,...
5296,R2XWWTSNR8IA,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,4
5297,W21VXDV1GVQR,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,4
5298,ZSN3Z3H8C98M,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,0
5299,0W4PAWWUMGDC,/home/is/dwipraseetyo-a/NAS_HAI/Datasets/combi...,4


In [43]:
df_reasoning.rename(columns={"idx_col": "user_id"})[["user_id", "formatted_llm_analyze_image"]]

,user_id,formatted_llm_analyze_image
0,ORU18QFQ7EFH,"The chest X-ray demonstrates diffuse, bilatera..."
1,IX37B4ZUB1IQ,"The chest X-ray shows diffuse, bilateral inter..."
2,Q6JS8BS4MRD9,The chest X-ray findings are suggestive of COV...
3,988GUD9IGCM1,The chest X-ray shows a relatively clear lung ...
4,2DK7ZQDS0M4I,"The chest X-ray demonstrates diffuse, bilatera..."
...,...,...
35593,1GOVQY01H4M3,The chest X-ray findings are highly suggestive...
35594,RZY9JRW4JK4R,The chest X-ray findings are highly suggestive...
35595,BKZ6GYHYEV2Y,The chest X-ray findings are suggestive of Tub...
35596,0BTN61W0Y4LY,The chest X-ray findings are highly suggestive...


In [64]:
DATA_PATH = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets"
for split in ['train', 'val', 'test']:
    df_reasoning = pd.read_csv(f"{DATA_PATH}/xray_medgemma.csv.{split}")
    df_original = pd.read_csv(f"{DATA_PATH}/xray_metadata_llm.csv.{split}")
    cols_to_drop = ["llm_analyze_image_x", "llm_analyze_image_y", "llm_analyze_image"]
    df_original = df_original.drop(columns=[c for c in cols_to_drop if c in df_original.columns]) 

    df_joined = df_original.merge(
        df_reasoning[["idx_col", "formatted_llm_analyze_image"]].rename(columns={"idx_col": "user_id"}), 
        on="user_id", 
        how="left"
    )
    
    df_joined.to_csv(f"{DATA_PATH}/xray_metadata_llm_final.csv.{split}", index=False)